<a href="https://colab.research.google.com/github/MBENALLA/dailycoins/blob/main/dailycoins.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json, pprint
import pandas as pd
from datetime import date, datetime
from sqlalchemy import create_engine
import schedule, time
from datetime import timezone
from apscheduler.schedulers.blocking import BlockingScheduler

In [2]:
url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'

In [3]:
parameters = {
    'start':'1',
    'limit':'150',
    'convert':'USD'
}

In [4]:
headers = {
    'Accepts':'application/json',
    'X-CMC_PRO_API_KEY':'281cb5da-359e-477c-8402-317a827916cd'
}

In [5]:
engine = create_engine('sqlite:///topdailycoins.db', echo = False)
#engine.execute("DELETE FROM dsc_dailycoins")
#engine.execute("DELETE FROM asc_dailycoins")
session = Session()

In [6]:
def getdailycoins():
    session.headers.update(headers)
    try:
        response = session.get(url, params=parameters)
        data = json.loads(response.text)
        coins = data['data']
        df = pd.DataFrame(columns=('symbol', 'percent_change_24h', 'percent_change_7d', 'date'))
        today = date.today().strftime("%d/%m/%Y")
        #pprint.pprint(coins)
        for x in coins:
            symbol = x['symbol']
            percent_change_24h = float(x['quote']['USD']['percent_change_24h'])
            percent_change_7d = float(x['quote']['USD']['percent_change_7d'])
            y  = pd.DataFrame([[symbol, percent_change_24h, percent_change_7d, today]],
                          columns=('symbol', 'percent_change_24h', 'percent_change_7d', 'date'))
            df = df.append(y, ignore_index=True) 
        df_dsc = df
        df_asc = df
        df_dsc.sort_values(by=['percent_change_24h'], inplace=True, ascending=False)
        df_asc.sort_values(by=['percent_change_24h'], inplace=True, ascending=True)
        top10coins_dsc = df_dsc[1:10]
        top10coins_asc = df_asc[1:10]
        top10coins_dsc.to_sql('dsc_dailycoins', con=engine, if_exists='append', index=False)
        top10coins_asc.to_sql('asc_dailycoins', con=engine, if_exists='append', index=False)
        #df1 = pd.read_sql('dsc_dailycoins', engine)
        #df2 = pd.read_sql('asc_dailycoins', engine)
        #print(df1)
        #print(df2)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        pprint.pprint(e)

In [11]:
def job():
    print('script ran at ', datetime.now())
    schedule.every(12).hours.do(getdailycoins)
    while True:
        schedule.run_pending()
        time.sleep(1)

In [ ]:
scheduler = BlockingScheduler()

scheduler.add_job(job, 'date', run_date='2021-12-04 05:00:00')

scheduler.start()